In [0]:
import numpy as np
import math
import pprint
from keras import layers, models, optimizers, regularizers
import tensorflow as tf
import keras
from collections import namedtuple
import csv

population_size = 10            #how many members are there per generation
network_depth = 5               #total number of layers in network excluding last fully connected layer and the flattening layer and any batch normalizations and activations
classes = 10                    #how many classes are there in the dataset
surviving_percentage = 0.5      #how many members of a generation survive and move over to the next generation (currently not used)
mutation_probability = 0.1      #how likely does an entry in the DNA randomly mutate (increment/decrement by 1)
generations = 5                 #after how many generations does the simulation end
input_shape = 32                #describes the size of the input image
channel_num = 3
update_type = 2

pp = pprint.PrettyPrinter(indent=4)

#define hyperparameters for the NN
learning_rate = 0.001                                                                                           #learning rate of the model
adm = keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #optimizer used
lossfunct = 'categorical_crossentropy'                                                                          #lossfunction during backpropagation
batch_size = 128                                                                                                #batchsize
epochs = 10                                                                                                     #training epochs before evaluation
validation_split_nn = 0.3                                                                                       #validation split of training data

#setup the population
def population_setup():
    population = np.empty((1 + network_depth  *3, population_size)).astype(int)                             #setup an empty matrix to be filled with the members of one generation of the population
    population[0,:] = 2 * np.random.rand(1, population_size)                                                #first dna entry: layer where the switch from feature extraction to fully connected occurs
    for i in range(0,network_depth):                                                                        #each subsequent DNA entry has its own meaning. A group of 3 entries determines the phenotype of a layer
        population[1 + 3 * i,:] = np.floor(3 * np.random.rand(1, population_size))                          #first entry in the block of three determies the layer type and ranges from 0 to 2
        population[2 + 3 * i,:] = np.floor(10 * np.random.rand(1, population_size))                         #second entry determines a parameter of the layer and ranges from 1 to 10
        population[3 + 3 * i,:] = np.floor(4 * np.random.rand(1, population_size))                          #third entry determines a second parameter and ranges from 0 to 3
    return population                                                                                       #return matrix filled with the DNA of every member of its population

#update the population to the next generation via mGA (keep best remove worst shuffle rest)
def population_update_mga(gen):
    max_score=0
    min_score=1
    max_ind = None
    min_ind = None
    for i in range(0,population_size):
        if(population_history[10 * (gen) + i]['fitness'] < min_score):
            min_score = population_history[10 * (gen) + i]['fitness']
            min_ind = i
        if(population_history[10 * (gen) + i]['fitness'] > max_score):
            max_score = population_history[10 * (gen) + i]['fitness']
            max_ind = i
    
    population = np.empty((1 + network_depth * 3, population_size)).astype(int)         #initialize an empty matrix to fit each members DNA
    population[:, 0] = population_history[10 * gen + max_ind]['dna']                    #first member: the fittest memeber of the previous generation survives
    split = (1 + 3 * np.floor(network_depth * np.random.rand())).astype(int)            #chose a random index where to perform the split for crossover (split along the lines of the layers)
    for i in range(1,population_size):                                                  #each subsequent member of the current generation is created in this forloop
        member1 = np.floor(np.random.rand() * population_size).astype(int)              #randomly select a member from the previous generation
        member2 = np.floor(np.random.rand() * population_size).astype(int)              #randomly select another member to mate with the first randomly selected member
        while member1 == max_ind or member1 == min_ind:                                 #if first selected member is the best or worst
            member1 = np.floor(np.random.rand() * population_size).astype(int)          #throw him out and generate another member (best is already in the current generation and worst dies off
        while member2 == max_ind or member2 == min_ind or member2 == member1:           #if the second selected member is best, worst or the same member was selected twice
            member2 = np.floor(np.random.rand() * population_size).astype(int)          #throw him out and generate another one. Then mate the two by combining them along the split
        population[:,i] = np.concatenate((population_history[10 * gen + member1]['dna'][:split], population_history[10 * gen + member2]['dna'][split:]))
    return population

#uptate via GA (shuffle all according to roulette wheel selection, mutate)
def population_update_ga(gen):
    population = np.empty((1 + network_depth * 3, population_size)).astype(int)         #initialize an empty matrix to fit each members DNA
    
    ###parent selection
    total_fitness = [0]                                                                 #initialize array to store cumulative fitness values
    for i in range(0,population_size):                                                  #iterate over the previous generation
        fitness = population_history[10 * (gen - 1) + i]['fitness']                     #read all fitness values from the previous generation into an array
        total_fitness.append(total_fitness[-1] + fitness)                               #accumulate all the fitness values from the previous generation
    for i in range(0,population_size):                                                  #each subsequent member of the current generation is created in this forloop
        split = (1 + 3 * np.floor(network_depth * np.random.rand())).astype(int)        #chose a random index where to perform the split for crossover (split along the lines of the layers)
        member1 = (np.random.rand() * total_fitness[-1])                                #randomly select a member from the previous generation
        j = 0                                                                           #variable for roulette wheel search
        while (member1 > total_fitness[j]):                                             #search which section of the roulette wheel the number falls in
            j += 1
        member1 = j - 1                                                                 #select that member as the first parent
        member2 = (np.random.rand() * total_fitness[-1])                                #randomly select another member to mate with the first randomly selected member
        k = 0                                                                           #second variable for the next parent
        while (member2 > total_fitness[k]):                                             #perform search on the roulette wheel
            k+=1
        member2 = k - 1                                                                 #set second parent
        while member2 == member1:                                                       #if the same member was selected twice
            member2 = (np.random.rand() * total_fitness[-1])                            #randomly select another member to mate with the first randomly selected member
            k = 0                                                                       #reinitialize the variable for the second parent
            while (member2 > total_fitness[k]):                                         #search on the roulette wheel
                k += 1
            member2 = k - 1                                                             #set the final second parent that is different from the first
        population[:,i] = np.concatenate((population_history[10 * gen + member1]['dna'][:split],population_history[10 * gen + member2]['dna'][split:]))

    ###mutation
    for j in range(0, 1 + 3 * network_depth):                     #begin mutation by iterating over the DNA of the member
        mutation = np.random.rand()                               #determine if that value shall mutate (change)
        if np.random.rand() <= mutation_probability:              #if mutation occurs
            inc_dec = np.floor(np.random.rand() * 2).astype(int)  #determine weither or not the value decrements by one or increments
            if inc_dec == 0:                                      #if it decrements
                population[j,i] -= 1
                if population[j,i] < 0:
                    population[j,i] += 2
            else:
                population[j,i] += 1
                if (j == 0 and population[j,i] > 1) or ((j - 1) % 3 == 0 and population[j,i] > 2) or ((j - 1) % 3 == 1 and population[j,i] > 9) or ((j - 1) % 3 == 2 and population[j,i] > 3):
                    population[j,i] -= 2                          #decrement
    return population                                                 #return the new generation

def population_update_ea(gen):
    global population_size
    new_pop_size = population_size - 2
    population = np.empty((1 + network_depth * 3,new_pop_size)).astype(int)             #initialize an empty matrix to fit each members DNA
    total_fitness = [0]                                                                 #initialize array to store cumulative fitness values
    for i in range(0,population_size):                                                  #iterate over the previous generation
        fitness = population_history[10 * (gen - 1) + i]['fitness']                     #read all fitness values from the previous generation into an array
        total_fitness.append(total_fitness[-1] + fitness)                               #accumulate all the fitness values from the previous generation
    for i in range(0,new_pop_size):                                                     #each subsequent member of the current generation is created in this forloop
        split = (1 + 3 * np.floor(network_depth * np.random.rand())).astype(int)        #chose a random index where to perform the split for crossover (split along the lines of the layers)
        member = (np.random.rand() * total_fitness[-1])                                 #randomly select a member from the previous generation
        j = 0                                                                           #initialize roulette wheel selector
        while (member > total_fitness[j]):                                              #check which member has been selected
            j += 1
        member = j - 1                                                                  #set the parent
        
        ###mutation
        population[:,i] = population_history[10 * gen + member1]['dna']
        for j in range(0, 1 + 3 * network_depth):                     #begin mutation by iterating over the DNA of the member
            mutation = np.random.rand()                               #determine if that value shall mutate (change)
            if np.random.rand() <= mutation_probability:              #if mutation occurs
                inc_dec = np.floor(np.random.rand() * 2).astype(int)  #determine weither or not the value decrements by one or increments
                if inc_dec == 0:                                      #if it decrements
                    population[j,i] -= 1
                    if population[j,i] < 0:
                        population[j,i] += 2
                else:
                    population[j,i] += 1
                    if (j == 0 and population[j,i] > 1) or ((j - 1) % 3 == 0 and population[j,i] > 2) or ((j - 1) % 3 == 1 and population[j,i] > 9) or ((j - 1) % 3 == 2 and population[j,i] > 3):
                        population[j,i] -= 2                          #decrement
    population_size -= 2
    return population                                             #return the new generation

#import data from mnist
def Import_data():
    cifar = tf.keras.datasets.cifar10
    (x_train, y_train), (x_test, y_test) = cifar.load_data()
    
    #turn labels into one-hot-encoded vectors
    y_train = keras.utils.to_categorical(y_train, classes)
    y_test = keras.utils.to_categorical(y_test, classes)
    
    # Normalize data
    x_train, x_test = x_train / 255.0, x_test / 255.0
    
    # Create dataset object, which controls all the data
    data1 = namedtuple("data", 'x_train y_train x_test y_test')
    params = data1(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)
    return params

#convert DNA to a model
def convert_to_pheno(dna,x_train,x_test,y_train,y_test):
    total_pooling_stride = 7                        #the total pooling stride must be kept count of (if total pooling is greater than the image size an error is thrown)
    model = models.Sequential()                     #instantiate model
    phenotype = ""                                  #instantiate description of phenotype of member
    flatten_layer = dna[0] * 3 + 1 + 3 * (network_depth - 2)
    for i in range(1,1 + 3 * network_depth,3):      #for each layer described in the DNA
        
        if i == flatten_layer:                      #if layer is the layer at which a flattening should occur
            model.add(layers.Flatten())             #add the flattening layer
            phenotype += "Flatten => "              #add description of phenotype
        
        #first type of layer: 0 => conv or dense
        if dna[i] == 0:
            if i < flatten_layer:                                                       #and if the layer is in the feature extraction segment
                kernel_size = 2 + dna[i + 2] + 1                                        #copy kernel size to adjust it
                while total_pooling_stride*(kernel_size) > input_shape:                 #here the total stride is checked and determined for validity
                    kernel_size -= 2                                                    #decrement kernel size until it fits with the feature map
                if kernel_size > 0:
                    model.add(layers.Conv2D(2 * (dna[i + 1] + 1), (kernel_size,kernel_size)))   #add a conv2d (using two subsequent genes for details)
                    model.add(layers.BatchNormalization())                                      #add batch norm
                    model.add(layers.Activation('relu'))                                        #add relu activation and add layer to phenotype description
                    phenotype += "Conv2d: Filters: " + str(2 * (dna[i + 1] + 1)) + " Kernel Size: " + str(kernel_size)  + " => "
            else:                                                                       #if the layer is in the second segment
                model.add(layers.Dense(10 * (dna[i + 1] + 2)))                          #add a dense layer (using subsequent gene for details)
                phenotype += "Dense: Output: " + str(10 * (dna[i + 1] + 2)) + " => "    #add layer to phenotype description
                if dna[i + 2] % 2 == 0:                                                 #use next gene to determine the activation
                    model.add(layers.Activation('sigmoid'))                             #sigmoid activation
                    phenotype += "Activation: Sigmoid"  + " => "                        #add layer to phenotype description
                else:                                                                   #or
                    model.add(layers.Activation('tanh'))                                #hyperbolic tangent
                    phenotype += "Activation: Tanh"  + " => "                           #add layer to phenotype description
        
        #second type of layer: 1 => dropout
        if dna[i] == 1:
            model.add(layers.Dropout(0.05 * (dna[i + 1] + 1)))                        #add dropout layer using subsequent gene to determine parameters (ignore second parameters, this one doesn't express in the phenotype
            phenotype += "Dropout: rate: " + str(0.05 * (dna[i + 1] + 1))  + " => "   #add layer to phenotype description
        
        #third type of layer: 2 => pooling or Regularization
        if dna[i] == 2:
            if i < flatten_layer:                                                 #and if the layer is in the feature extraction segment
                stride = 2 + dna[i + 2] + 1                                       #copy kernel size to adjust it
                while total_pooling_stride*(stride) > input_shape:                #here the total stride is checked and determined for validity
                    stride -= 1
                if stride > 1:
                    total_pooling_stride *= 2 + dna[i + 1]                                  #if it is a valid stride add it to the total stride
                    if dna[i + 2] - 2 < 0:                                                  #determine the type of pooling
                        model.add(layers.MaxPooling2D((2 + dna[i + 1],2 + dna[i + 1])))     #max pooling (subsequent gene for parameters, second parameter doesnt express in phenotype
                        phenotype += "Maxpool2D: stride: " + str(2 + dna[i + 1])  + " => "  #add layer to phenotype description
                    else:
                        model.add(layers.AveragePooling2D((2 + dna[i + 1],2 + dna[i + 1])))
                        phenotype += "Avgpool2D: stride: " + str(2 + dna[i + 1])  + " => "
            else:
                model.add(keras.layers.GaussianNoise(dna[i + 1] + 1 / 10.))
                phenotype += "GaussianNoise: Std dev: " + str(dna[i + 1] + 1 / 10.) + " => "
    model.add(layers.Dense(classes))                                                    #finally add a dense layer, with the nuber of classes as an output
    model.add(layers.Activation('softmax'))                                             #perform a softmax
    phenotype += " Dense: Output: " + str(classes)
    model.compile(loss=lossfunct, optimizer=adm, metrics=['accuracy'])                  #compile model with Adam optimizer
    history = model.fit(x_train, y_train, batch_size, epochs, validation_split=validation_split_nn, verbose=0)
    score = model.evaluate(x_test, y_test, verbose=0)                                   #evaluate on the testset
    print(score[1])
    return score, phenotype                                                             #return score from the evaluation of the testset


In [0]:
if __name__ == '__main__':
    population_history = []                                                                             #instantiate a JSON object to hold the data on the entire population
    params = Import_data()                                                                              #import the data from CIFAR100
    pop = population_setup()                                                                            #setup the first gen of the population
    for j in range(0,generations):
        print("")
        print("#####################")
        print("generation: " + str(j))
        print(pop)
        best = None
        max_score = 0
        min_score = 1
        worst = None
        for i in range(0,pop.shape[1]):                                                                 #go through the population
            exists = None
            for item in range(0,len(population_history)):                                               #check entire pop history to verify that the member hadn't previously existed
                if np.array_equal(population_history[item]['dna'],pop[:,i]):                            #if the member had already existed, there is no need to compute its fitness value
                    exists = item
                    break
            if exists is None:                                                                          #if the member hasn't existed yet
                score, pheno = convert_to_pheno(pop[:,i],**params._asdict())                            #determine its fitness and phenotype
                population_history.append({'dna':pop[:,i], 'fitness':score[1], 'phenotype':pheno, 'generation':j})      #append the member to the history
            else:                                                                                       #if member has already existed
                old_member = population_history[item]                                                   #pull up the member from the record
                population_history.append(old_member)                                                   #append it to the history again for consistency
                population_history[-1]['generation'] = j
                print(old_member['fitness'])
    
        #update the population
        print("best score: " + str(max_score) + ", " + str(best))                                       #print best member and his score
        pp.pprint(population_history)
        if update_type == 0:
            pop = population_update_mga(j)
        elif update_type == 1:
            pop = population_update_ga(j)
        elif update_type == 2:
            pop = population_update_ea(j)

#write results to csv file
with open(str(update_type) + 'GA' + str(population_size) + '.csv', mode='w') as csv_file:
    fieldnames = ['fitness', 'generation','phenotype','dna']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
        for i in range(0,len(population_history)):
            writer.writerow(population_history[i])




#####################
generation: 0
[[0 0 1 0 1 1 0 0 1 1]
 [1 1 0 2 1 0 0 0 0 0]
 [4 4 7 4 3 7 9 0 1 9]
 [0 3 2 1 1 0 3 0 1 1]
 [1 0 1 1 0 2 1 2 2 1]
 [3 5 1 1 7 8 3 9 7 0]
 [0 2 1 2 0 1 0 2 0 0]
 [0 1 2 0 2 0 1 2 0 2]
 [3 3 5 1 8 6 2 6 6 5]
 [0 3 3 0 3 3 0 1 1 0]
 [1 2 2 2 0 2 0 2 1 1]
 [4 0 4 0 7 4 4 5 3 0]
 [0 2 0 2 2 2 3 0 0 3]
 [2 1 0 2 0 0 0 2 2 2]
 [1 3 5 6 2 3 2 3 1 4]
 [1 1 3 0 1 1 0 2 0 3]]
0.2954
0.4629
0.6124
0.1245
0.3917
0.4937
0.4639
0.2332
0.416
0.4965
[   {   'dna': array([0, 1, 4, 0, 1, 3, 0, 0, 3, 0, 1, 4, 0, 2, 1, 1]),
        'fitness': 0.2954,
        'generation': 0,
        'phenotype': 'Dropout: rate: 0.25 => Dropout: rate: 0.2 => Conv2d: '
                     'Filters: 8 Kernel Size: 1 => Flatten => Dropout: rate: '
                     '0.25 => GaussianNoise: Std dev: 1.1 =>  Dense: Output: '
                     '10'},
    {   'dna': array([0, 1, 4, 3, 0, 5, 2, 1, 3, 3, 2, 0, 2, 1, 3, 1]),
        'fitness': 0.4629,
        'generation': 0,
        'pheno